In [8]:
import sys

if str(type(sys.stdout)) == "<class 'ipykernel.iostream.OutStream'>":
    saved_std = sys.stdout
else:
    sys.stdout = saved_std

sys.stdout

### TO DO:
1. ~~Cleanup imports~~
1. ~~Decide on how to impute values for each column~~
1. ~~Use OHE and binarize the data.~~
1. ~~Make pipelines that process the data~~
1. Use pipelines to fit models

In [34]:
#models:
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn import tree
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from xgboost import XGBClassifier


#Preprocessing:
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer


#Other:
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline
from sklearn import metrics
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [18]:
df = pd.read_pickle('Data/modifiedData.pickle')
test = pd.read_pickle('Data/modifiedTest.pickle')
y = df['Survived']
df.drop('Survived',axis=1,inplace=True)
display(df)
display(test)

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Title,Deck,hasNChildren,hasCabin,has1or2sib,has3PSib
PassengerId,,,,,,,,,,,,,
1,3,male,22.0,1,0,7.2500,S,Mr.,U,0,0,1,0
2,1,female,38.0,1,0,71.2833,C,Mrs.,C,0,1,1,0
3,3,female,26.0,0,0,7.9250,S,Miss.,U,0,0,0,0
4,1,female,35.0,1,0,53.1000,S,Mrs.,C,0,1,1,0
5,3,male,35.0,0,0,8.0500,S,Mr.,U,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
887,2,male,27.0,0,0,13.0000,S,Mr.,U,0,0,0,0
888,1,female,19.0,0,0,30.0000,S,Miss.,B,0,1,0,0
889,3,female,15.0,1,2,23.4500,S,Miss.,U,0,0,1,0


,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Title,Deck,hasNChildren,hasCabin,has1or2sib,has3PSib
PassengerId,,,,,,,,,,,,,
892,3,male,34.5,0,0,7.8292,Q,Mr.,U,0,0,0,0
893,3,female,47.0,1,0,7.0000,S,Mrs.,U,0,0,1,0
894,2,male,62.0,0,0,9.6875,Q,Mr.,U,0,0,0,0
895,3,male,27.0,0,0,8.6625,S,Mr.,U,0,0,0,0
896,3,female,22.0,1,1,12.2875,S,Mrs.,U,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1305,3,male,37.0,0,0,8.0500,S,Mr.,U,0,0,0,0
1306,1,female,39.0,0,0,108.9000,C,Mr.,C,0,1,0,0
1307,3,male,38.5,0,0,7.2500,S,Mr.,U,0,0,0,0


In [ ]:
# Age & Fare will be filled in using the appropriate distribution (as normal doesn't fit).
# SibSp & Parch will be filled in as 0
# Pclass & Embarked will be filled in with modal class (3rd) 

In [27]:
preSib = Pipeline(
    [('Sib_NA', SimpleImputer(strategy='constant', fill_value=0)),('Scaler',StandardScaler())])

preClass = Pipeline([('Class_NA', SimpleImputer(
    strategy='most_frequent')), ("OHE", OneHotEncoder(handle_unknown='ignore'))])

# handle_unknown='ignore' means that if the OHE encounters a class it doesn't recognize when transforming,
# it'll continue instead of throwing an error.

Preprocess = ColumnTransformer([('Scaler',StandardScaler(),
                                ['Age', 'Fare', 'hasNChildren', 'hasCabin', 'has1or2sib', 'has3PSib']),
                                ('FillNA', preSib, ['SibSp', 'Parch']), 
                                ('OHE',preClass,['Sex','Deck','Title','Embarked','Pclass'])])

The models I'm using are those suggested [here](https://www.kaggle.com/kenjee/titanic-project-example)

In [28]:
gnb_pipe = Pipeline([('Preprocess',Preprocess),('GNB',GaussianNB())])
cv = cross_val_score(gnb_pipe,df,y,cv=5)
print(cv.mean())

0.7677170296905406


In [29]:
tree_pipe = Pipeline([('Preprocess',Preprocess),('Tree',tree.DecisionTreeClassifier(random_state=7))])
cv = cross_val_score(tree_pipe,df,y,cv=5)
print(cv.mean())

0.7620802209528592


In [33]:
SVC_pipe = Pipeline([('Preprocess',Preprocess),('SVC',SVC(probability = True))])
cv = cross_val_score(SVC_pipe,df,y,cv=5)
print(cv.mean()) # Maybe try one with less overlapping features.

0.827154604230745


In [37]:
xgb_pipe = Pipeline([('Preprocess',Preprocess),
                     ('xgb',XGBClassifier(random_state=7,use_label_encoder=False,eval_metric='logloss'))])
cv = cross_val_score(xgb_pipe,df,y,cv=5)
print(cv.mean()) # Maybe try one with less overlapping features.

0.8114744837110036


In [38]:
RF_pipe = Pipeline([('Preprocess',Preprocess),('RF',RandomForestClassifier(random_state=7))])
cv = cross_val_score(RF_pipe,df,y,cv=5)
print(cv.mean())

0.8024731655263324


In [39]:
KNN_pipe = Pipeline([('Preprocess',Preprocess),('KNN',KNeighborsClassifier())])
cv = cross_val_score(KNN_pipe,df,y,cv=5)
print(cv.mean())

0.7957504237022157


In [40]:
logreg_pipe = Pipeline([('Preprocess',Preprocess),('Logreg',LogisticRegression())])
cv = cross_val_score(logreg_pipe,df,y,cv=5)
print(cv.mean())

0.8249325214989642


In [41]:
voting_clf = VotingClassifier(estimators = [('lr',logreg_pipe),('knn',KNN_pipe),('rf',RF_pipe),
                                            ('svc',SVC_pipe),('xgb',xgb_pipe)], voting = 'soft')
cv = cross_val_score(voting_clf,df,y,cv=5)
print(cv.mean())

0.8327851358985626


# OLD STUFF, PROLLY GONNA DELETE

In [ ]:
def print_best(Grid,Prints = True):
    if Prints:
        print(Grid.best_score_,Grid.best_params_)
    if not Prints:
        return [Grid.best_score_,Grid.best_params_]

In [ ]:
Forest_Grid_NR = GridSearchCV(Pipe_Forest,param_grid=params_Frst,scoring='f1',cv=10,return_train_score=True)

Grad_Grid_NR = GridSearchCV(Pipe_Grad,param_grid=params_Grad,scoring='roc_auc',cv=10,return_train_score=True)

Grad_Grid_NR.fit(data,y)
print_best(Grad_Grid_NR)

Forest_Grid_NR.fit(data,y)
print_best(Forest_Grid_NR)

In [ ]:
Forest_Grid_OF = GridSearchCV(Pipe_Forest,param_grid=params_Frst,scoring='accuracy',cv=5,return_train_score=True)
Grad_Grid_OF = GridSearchCV(Pipe_Grad,param_grid=params_Grad,scoring='accuracy',cv=5,return_train_score=True)

Grad_Grid_OF.fit(data,y)
print_best(Grad_Grid_OF)

Forest_Grid_OF.fit(data,y)
print_best(Forest_Grid_OF)

In [ ]:
Forest_pred_OF = pd.Series(Forest_Grid_OF.predict(test), index=test.index, name='Survived')
Grad_pred_OF = pd.Series(Grad_Grid_OF.predict(test), index=test.index, name='Survived')

print(metrics.confusion_matrix(y,Forest_Grid_OF.predict(data)))
print(metrics.confusion_matrix(y,Grad_Grid_OF.predict(data)))


Forest_pred_OF.to_csv('Predictions/Forest_Pred_OF_1.csv')
Grad_pred_OF.to_csv('Predictions/Grad_Pred_OF_1.csv')


# Confusion Matrix:
#    | 0  | 1  |
#  0 | TN | FN |
#  1 | FP | TP |

In [ ]:
Forest_pred_NR = pd.Series(Forest_Grid_NR.predict(test), index=test.index, name='Survived')
Grad_pred_NR = pd.Series(Grad_Grid_NR.predict(test), index=test.index, name='Survived')

print(metrics.confusion_matrix(y,Forest_Grid_NR.predict(data)))
print(metrics.confusion_matrix(y,Grad_Grid_NR.predict(data)))

Forest_pred_NR.to_csv('Predictions/Forest_Pred_NR_1.csv')
Grad_pred_NR.to_csv('Predictions/Grad_Pred_NR_1.csv')

In [ ]:
params_Grad_f1 = dict(Gradient__n_estimators = [100,400,600],
                   Gradient__max_depth=[2,5], 
                   Gradient__random_state=[42],
                   Gradient__learning_rate=[0.2,0.1,0.05])

# params_Grad_f1 = dict(Gradient__n_estimators = [400],
#                    Gradient__max_depth=[2], 
#                    Gradient__random_state=[42],
#                    Gradient__learning_rate=[0.1])

params_Frst_f1 = dict(Forest__n_estimators=[200,500,700],
                   Forest__max_depth=[None,6,9,12],
                   Forest__random_state=[42])


In [61]:

Grad_Grid_f1 = GridSearchCV(Pipe_Grad,param_grid=params_Grad_f1,scoring='f1',cv=5,verbose=3,n_jobs=1)
Forest_Grid_f1 = GridSearchCV(Pipe_Forest,param_grid=params_Frst_f1,scoring='f1',cv=5,verbose=3,n_jobs=1)

sys.stdout = open(1, 'w')
Grad_Grid_f1.fit(data,y)
print('Done with grid :)')
sys.stdout = saved_std

print_best(Grad_Grid_f1)

sys.stdout = open(1, 'w')
Forest_Grid_f1.fit(data,y)
print('Done with grid :)')
sys.stdout = saved_std

print_best(Forest_Grid_f1)

0.7591260150442718 {'Gradient__learning_rate': 0.1, 'Gradient__max_depth': 2, 'Gradient__n_estimators': 400, 'Gradient__random_state': 42}
0.7551045173645473 {'Forest__max_depth': 12, 'Forest__n_estimators': 700, 'Forest__random_state': 42}


In [62]:
Grad_pred_f1 = pd.Series(Grad_Grid_f1.predict(test), index=test.index, name='Survived')
Forest_pred_f1 = pd.Series(Forest_Grid_f1.predict(test), index=test.index, name='Survived')

print(metrics.confusion_matrix(y,Grad_Grid_f1.predict(data)))
print(metrics.confusion_matrix(y,Forest_Grid_f1.predict(data)))


Grad_pred_f1.to_csv('Predictions/Grad_Pred_f1.csv')
Forest_pred_f1.to_csv('Predictions/Forest_Pred_f1.csv')


[[517  32]
 [ 57 285]]
[[543   6]
 [ 27 315]]


In [ ]:
Tree = tree.DecisionTreeClassifier(random_state=32)
Pipe_Tree = Pipeline([
    ('Preprocessing', Preprocess),
    ('Tree', Tree)
])

params_Tree = dict(Tree__max_depth = [None],
                   Tree__splitter = ['best','random'],
                   Tree__min_samples_split = range(2,4),
                   Tree__min_samples_leaf = range(1,4),
                   Tree__max_leaf_nodes = [None] + list(range(150,300,50))
                   )


In [64]:
Tree_Grid = GridSearchCV(Pipe_Tree,param_grid=params_Tree,scoring='f1',cv=10,verbose=3)

sys.stdout = open(1, 'w')
Tree_Grid.fit(data,y)
print('Done with grid :)')
sys.stdout = saved_std

print_best(Tree_Grid)
print(metrics.confusion_matrix(y,Tree_Grid.predict(data)))

0.7534598724095104 {'Tree__max_depth': None, 'Tree__max_leaf_nodes': 200, 'Tree__min_samples_leaf': 2, 'Tree__min_samples_split': 2, 'Tree__splitter': 'random'}
[[521  28]
 [ 76 266]]


In [ ]:
logreg = LogisticRegression(random_state=32)
Pipe_logreg = Pipeline([
    ('Preprocessing', Preprocess),
    ('logreg', logreg)
])

params_logreg = dict(logreg__penalty = ['l1','l2'],
                     logreg__solver = ['liblinear'],
                     logreg__max_iter = [100,125,75],
                     logreg__C = np.arange(1,2,0.05),
                     logreg__class_weight = ['balanced',None]
                  )

In [82]:
logreg_Grid = GridSearchCV(Pipe_logreg,param_grid=params_logreg,scoring='f1',cv=10,verbose=3)

sys.stdout = open(1, 'w')
logreg_Grid.fit(data,y)
print('Done with grid :)')
sys.stdout = saved_std

print_best(logreg_Grid)
print(metrics.confusion_matrix(y,logreg_Grid.predict(data)))

0.734546590750368 {'logreg__C': 1.1, 'logreg__class_weight': 'balanced', 'logreg__max_iter': 100, 'logreg__penalty': 'l1', 'logreg__solver': 'liblinear'}
[[437 112]
 [ 73 269]]


In [ ]:
Extra_For = ExtraTreesClassifier(warm_start=True,random_state=1)
Pipe_Extra = Pipeline([
    ('Preprocessing', Preprocess),
    ('Extra', Extra_For)
])

In [ ]:
params_Extra = dict(Extra__n_estimators = [50,100,200],
                    Extra__max_depth = [None],
                    Extra__class_weight = [None],
                    Extra__ccp_alpha = [0.0]
                    )

In [35]:
sys.stdout = open(1, 'w')
Extra_Grid_f1 = GridSearchCV(Pipe_Extra,param_grid=params_Extra,scoring='f1',cv=10,n_jobs=1,verbose=3)
Extra_Grid_f1.fit(data,y)
print('Done with grid :)')
sys.stdout = saved_std

print_best(Extra_Grid_f1)
print(metrics.confusion_matrix(y,Extra_Grid_f1.predict(data)))
pd.Series(Extra_Grid_f1.predict(test), index=test.index, name='Survived').to_csv('Predictions/Extra_Pred_f1.csv')

0.7321402121164768 {'Extra__ccp_alpha': 0.0, 'Extra__class_weight': None, 'Extra__max_depth': None, 'Extra__n_estimators': 100}


In [ ]:
Ada = AdaBoostClassifier(random_state=1,base_estimator = tree.DecisionTreeClassifier(random_state=1))
Pipe_Ada = Pipeline([
    ('Preprocessing', Preprocess),
    ('Ada', Ada)
])
# possible models for Ada:
# ExtraTrees
# 

In [ ]:
params_Ada = dict(
                  Ada__n_estimators = [50,100,200]
                  )

In [88]:
sys.stdout = open(1, 'w')
Ada_Grid_f1 = GridSearchCV(Pipe_Ada,param_grid=params_Ada,cv=10,verbose=3,scoring='f1')
Ada_Grid_f1.fit(data,y)
print('Done with grid :)')
sys.stdout = saved_std

print_best(Ada_Grid_f1)
print(metrics.confusion_matrix(y,Ada_Grid_f1.predict(data)))
pd.Series(Ada_Grid_f1.predict(test), index=test.index, name='Survived').to_csv('Predictions/Ada_Pred_f1.csv')

0.744563123023356 {'Ada__n_estimators': 50}
[[539  10]
 [  7 335]]


## Bests models: 
1. GradientBoosting (f1)
2. RandomForest (f1)

### Now to feature engineering
Our features should include these facts:
1. Kids and teens have a higher chance of surviving
2. People with family have a higher chance of surviving

In [ ]:
data['Age'] = Nmrcl_trns.fit_transform(data[['Age']])
data['Has_Someone'] = Binarizer(threshold=0.9).transform([data['Parch'] + data['SibSp']])[0]
data['Has_SibSp'] = Binarizer(threshold=0.9).transform([data['SibSp']])[0]
data['Under_17'] = 1 - Binarizer(threshold=17).transform([(data['Age'])])[0]
data

In [ ]:
test['Age'] = Nmrcl_trns.fit_transform(test[['Age']])
test['Has_Someone'] = Binarizer(threshold=0.9).transform([test['Parch'] + test['SibSp']])[0]
test['Has_SibSp'] = Binarizer(threshold=0.9).transform([test['SibSp']])[0]
test['Under_17'] = 1 - Binarizer(threshold=17).transform([(test['Age'])])[0]
test

In [ ]:
Feature_Pairs = [
    (['Sex','Pclass','Embarked'],['SibSp','Age','Fare']),
    (['Sex','Pclass','Embarked','Under_17','Has_SibSp'],['Fare']),
    (['Sex','Pclass','Embarked','Under_17','Has_SibSp'],[]),
    (['Sex','Pclass','Embarked','Has_SibSp'],['Age']),
    (['Sex','Pclass','Embarked','Under_17','Has_Someone'],[]),
    (['Sex','Pclass','SibSp','Embarked'],['Age','Fare']),
    (['Sex','Pclass','SibSp','Embarked','Under_17','Has_Someone'],['Age','Fare']),
    (['Sex','Pclass','Embarked','Under_17'],['SibSp','Age','Fare']),
]

scores = []

In [138]:
for pair in Feature_Pairs[0:2]:
    Preprocess2 = ColumnTransformer([('Categorical_cols', Ctgrcl_trns,pair[0]),
                                 ('Numrecial_cols',Nmrcl_trns,pair[1])])
    
    Grad_Fin = GradientBoostingClassifier(warm_start=True)
    Pipe_Grad_Fin = Pipeline([
        ('Preprocessing', Preprocess2),
        ('Gradient', Gradient)
    ])

    Forest_Fin = RandomForestClassifier(warm_start=True)
    Pipe_Forest_Fin = Pipeline([
        ('Preprocessing', Preprocess2),
        ('Forest', Forest)
    ])
    
    Grad_Grid_Fin = GridSearchCV(Pipe_Grad_Fin,param_grid=params_Grad_f1,scoring='f1',cv=10,verbose=10,n_jobs=1)
    Forest_Grid_Fin = GridSearchCV(Pipe_Forest_Fin,param_grid=params_Frst_f1,scoring='f1',cv=10,verbose=10,n_jobs=1)

    sys.stdout = open(1, 'w')
    Grad_Grid_Fin.fit(data,y)
    print('Done with grid :)')
    sys.stdout = saved_std
    scores.append([print_best(Grad_Grid_Fin,Prints=False)])
    
    print_best(Grad_Grid_Fin)

    sys.stdout = open(1, 'w')
    Forest_Grid_Fin.fit(data,y)
    print('Done with grid :)')
    sys.stdout = saved_std
    scores.append([print_best(Forest_Grid_Fin,Prints=False)])

    print_best(Forest_Grid_Fin)
    print("\n",pair)

0.7702513487952218 {'Gradient__learning_rate': 0.05, 'Gradient__max_depth': 2, 'Gradient__n_estimators': 600, 'Gradient__random_state': 42}
0.7731677336618445 {'Forest__max_depth': 9, 'Forest__n_estimators': 200, 'Forest__random_state': 42}

 (['Sex', 'Pclass', 'Embarked'], ['SibSp', 'Age', 'Fare'])
0.7732873120586929 {'Gradient__learning_rate': 0.1, 'Gradient__max_depth': 5, 'Gradient__n_estimators': 100, 'Gradient__random_state': 42}
0.7556297083572934 {'Forest__max_depth': 9, 'Forest__n_estimators': 500, 'Forest__random_state': 42}

 (['Sex', 'Pclass', 'Embarked', 'Under_17', 'Has_SibSp'], ['Fare'])


In [139]:
for pair in Feature_Pairs[2:-1]:
    Preprocess2 = ColumnTransformer([('Categorical_cols', Ctgrcl_trns,pair[0]),
                                 ('Numrecial_cols',Nmrcl_trns,pair[1])])
    
    Grad_Fin = GradientBoostingClassifier(warm_start=True)
    Pipe_Grad_Fin = Pipeline([
        ('Preprocessing', Preprocess2),
        ('Gradient', Gradient)
    ])

    Forest_Fin = RandomForestClassifier(warm_start=True)
    Pipe_Forest_Fin = Pipeline([
        ('Preprocessing', Preprocess2),
        ('Forest', Forest)
    ])
    
    Grad_Grid_Fin = GridSearchCV(Pipe_Grad_Fin,param_grid=params_Grad_f1,scoring='f1',cv=10,verbose=10,n_jobs=1)
    Forest_Grid_Fin = GridSearchCV(Pipe_Forest_Fin,param_grid=params_Frst_f1,scoring='f1',cv=10,verbose=10,n_jobs=1)

    sys.stdout = open(1, 'w')
    Grad_Grid_Fin.fit(data,y)
    print_best(Grad_Grid_Fin)
    print('Done with grid :)')
    sys.stdout = saved_std
    scores.append([print_best(Grad_Grid_Fin,Prints=False)])

    print_best(Grad_Grid_Fin)

    sys.stdout = open(1, 'w')
    Forest_Grid_Fin.fit(data,y)
    print_best(Grad_Grid_Fin)
    print('Done with grid :)')
    sys.stdout = saved_std
    scores.append([print_best(Forest_Grid_Fin,Prints=False)])

    print_best(Forest_Grid_Fin)
    print("\n",pair)

0.7320755603515294 {'Gradient__learning_rate': 0.1, 'Gradient__max_depth': 2, 'Gradient__n_estimators': 400, 'Gradient__random_state': 42}
0.7216048015283008 {'Forest__max_depth': 6, 'Forest__n_estimators': 200, 'Forest__random_state': 42}

 (['Sex', 'Pclass', 'Embarked', 'Under_17', 'Has_SibSp'], [])
0.7549350622890174 {'Gradient__learning_rate': 0.2, 'Gradient__max_depth': 2, 'Gradient__n_estimators': 400, 'Gradient__random_state': 42}
0.7447045928248592 {'Forest__max_depth': 6, 'Forest__n_estimators': 200, 'Forest__random_state': 42}

 (['Sex', 'Pclass', 'Embarked', 'Has_SibSp'], ['Age'])
0.7266765437720577 {'Gradient__learning_rate': 0.05, 'Gradient__max_depth': 2, 'Gradient__n_estimators': 400, 'Gradient__random_state': 42}
0.7142252496161967 {'Forest__max_depth': None, 'Forest__n_estimators': 200, 'Forest__random_state': 42}

 (['Sex', 'Pclass', 'Embarked', 'Under_17', 'Has_Someone'], [])
0.7697356668170433 {'Gradient__learning_rate': 0.1, 'Gradient__max_depth': 2, 'Gradient__n_e

In [140]:
for pair in [Feature_Pairs[-1]]:
    Preprocess2 = ColumnTransformer([('Categorical_cols', Ctgrcl_trns,pair[0]),
                                 ('Numrecial_cols',Nmrcl_trns,pair[1])])
    
    Grad_Fin = GradientBoostingClassifier(warm_start=True)
    Pipe_Grad_Fin = Pipeline([
        ('Preprocessing', Preprocess2),
        ('Gradient', Gradient)
    ])

    Forest_Fin = RandomForestClassifier(warm_start=True)
    Pipe_Forest_Fin = Pipeline([
        ('Preprocessing', Preprocess2),
        ('Forest', Forest)
    ])
    
    Grad_Grid_Fin = GridSearchCV(Pipe_Grad_Fin,param_grid=params_Grad_f1,scoring='f1',cv=10,verbose=10,n_jobs=1)
    Forest_Grid_Fin = GridSearchCV(Pipe_Forest_Fin,param_grid=params_Frst_f1,scoring='f1',cv=10,verbose=10,n_jobs=1)

    sys.stdout = open(1, 'w')
    Grad_Grid_Fin.fit(data,y)
    print_best(Grad_Grid_Fin)
    print('Done with grid :)')
    sys.stdout = saved_std
    scores.append(print_best(Grad_Grid_Fin,Prints=False))

    print_best(Grad_Grid_Fin)

    sys.stdout = open(1, 'w')
    Forest_Grid_Fin.fit(data,y)
    print_best(Grad_Grid_Fin)
    print('Done with grid :)')
    sys.stdout = saved_std
    scores.append([print_best(Forest_Grid_Fin,Prints=False)])

    print_best(Forest_Grid_Fin)
    print("\n",pair)

0.7702513487952218 {'Gradient__learning_rate': 0.05, 'Gradient__max_depth': 2, 'Gradient__n_estimators': 600, 'Gradient__random_state': 42}
0.7710090271179103 {'Forest__max_depth': 12, 'Forest__n_estimators': 500, 'Forest__random_state': 42}

 (['Sex', 'Pclass', 'Embarked', 'Under_17'], ['SibSp', 'Age', 'Fare'])


In [ ]:
Best_Pairs = [(['Sex', 'Pclass', 'Embarked'], ['SibSp', 'Age', 'Fare']),
              (['Sex', 'Pclass', 'Embarked'], ['Parch','SibSp', 'Age', 'Fare'])]

In [ ]:
params_Grad_Fin = [
    dict(Gradient__n_estimators = [400,600,700],
         Gradient__max_depth=[2,3], 
         Gradient__random_state=[42],
         Gradient__learning_rate=[0.1,0.05]),
    dict(Gradient__n_estimators = [100,200],
         Gradient__max_depth=[8,9,10,12], 
         Gradient__random_state=[42],
         Gradient__learning_rate=[0.1,0.05])
                  ]


params_Frst_Fin = dict(Forest__n_estimators=[200,350,500],
                   Forest__max_depth=[None,8,9,10],
                   Forest__random_state=[42])


In [192]:
for index,pair in enumerate(Best_Pairs):
    Preprocess2 = ColumnTransformer([('Categorical_cols', Ctgrcl_trns,pair[0]),
                                 ('Numrecial_cols',Nmrcl_trns,pair[1])])
    
    Grad_Fin = GradientBoostingClassifier(warm_start=True)
    Pipe_Grad_Fin = Pipeline([
        ('Preprocessing', Preprocess2),
        ('Gradient', Gradient)
    ])

    Forest_Fin = RandomForestClassifier(warm_start=True)
    Pipe_Forest_Fin = Pipeline([
        ('Preprocessing', Preprocess2),
        ('Forest', Forest)
    ])
    
    Grad_Grid_Fin = GridSearchCV(Pipe_Grad_Fin,param_grid=params_Grad_Fin[index],scoring='f1',cv=10,verbose=10,n_jobs=1)
    Forest_Grid_Fin = GridSearchCV(Pipe_Forest_Fin,param_grid=params_Frst_Fin,scoring='f1',cv=10,verbose=10,n_jobs=1)

    sys.stdout = open(1, 'w')
    Grad_Grid_Fin.fit(data,y)
    print_best(Grad_Grid_Fin)
    print('Done with grid :)')
    sys.stdout = saved_std
    scores.append(print_best(Grad_Grid_Fin,Prints=False))

    print_best(Grad_Grid_Fin)
    print(metrics.confusion_matrix(y,Grad_Grid_Fin.predict(data)))
    pd.Series(Grad_Grid_Fin.predict(test), index=test.index, name='Survived'
             ).to_csv('Predictions/Best_Pred_Grad_{}.csv'.format(index + 1))
    
    sys.stdout = open(1, 'w')
    Forest_Grid_Fin.fit(data,y)
    print_best(Grad_Grid_Fin)
    print('Done with grid :)')
    sys.stdout = saved_std
    scores.append(print_best(Forest_Grid_Fin,Prints=False))

    print_best(Forest_Grid_Fin)
    print(metrics.confusion_matrix(y,Forest_Grid_Fin.predict(data)))
    pd.Series(Forest_Grid_Fin.predict(test), index=test.index, name='Survived'
             ).to_csv('Predictions/Best_Pred_Forest_{}.csv'.format(index + 1))
    
    print("\n",pair)

0.7702513487952218 {'Gradient__learning_rate': 0.05, 'Gradient__max_depth': 2, 'Gradient__n_estimators': 600, 'Gradient__random_state': 42}
[[519  30]
 [ 66 276]]
0.7731677336618445 {'Forest__max_depth': 9, 'Forest__n_estimators': 200, 'Forest__random_state': 42}
[[538  11]
 [ 44 298]]

 (['Sex', 'Pclass', 'Embarked'], ['SibSp', 'Age', 'Fare'])
0.7603701395235238 {'Gradient__learning_rate': 0.05, 'Gradient__max_depth': 9, 'Gradient__n_estimators': 100, 'Gradient__random_state': 42}
[[530  19]
 [ 49 293]]
0.7608696728957778 {'Forest__max_depth': 8, 'Forest__n_estimators': 500, 'Forest__random_state': 42}
[[527  22]
 [ 71 271]]

 (['Sex', 'Pclass', 'Embarked', 'Under_17', 'Has_SibSp'], ['Fare'])


In [27]:
Ctgrcl_Best = make_pipeline(SimpleImputer(strategy='most_frequent'),OneHotEncoder(),)
Nmrcl_Best = 
for index,pair in enumerate(Best_Pairs):
    Preprocess3 = ColumnTransformer([('Categorical_cols', Ctgrcl_Best,pair[0]),
                                 ('Numrecial_cols',Nmrcl_Best,pair[1])])

[]